# 📋 Sistema di Pianificazione Corsi 2026 - Notebook Autonomo

Questo notebook è completamente **autonom** e non dipende da file esterni Python.
Tutto il codice necessario è incluso inline.

## ✅ Prerequisiti
- Python 3.x
- openpyxl (per gestione Excel)
- reportlab (per generazione PDF)

## 🎯 Funzionalità
1. **Creazione file Excel** con pianificazione corsi
2. **Generazione PDF** per aule, formatori, corsi e report settimanali
3. **Apertura file** con applicazioni di sistema

---

In [ ]:
# Import librerie standard
import os
import subprocess
import platform
from datetime import datetime, timedelta
from pathlib import Path
from collections import defaultdict
import calendar

# Import librerie esterne (devono essere già installate)
try:
    from openpyxl import Workbook, load_workbook
    from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
    from openpyxl.utils import get_column_letter
    from openpyxl.worksheet.datavalidation import DataValidation
    from openpyxl.formatting.rule import CellIsRule, FormulaRule
    
    from reportlab.lib.pagesizes import A4, landscape
    from reportlab.lib.units import cm
    from reportlab.lib import colors
    from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer, PageBreak
    from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
    from reportlab.lib.enums import TA_CENTER, TA_LEFT, TA_RIGHT
    
    print("✅ Tutte le librerie importate con successo!")
except ImportError as e:
    print(f"❌ Errore import: {e}")
    print("\\n💡 Installa le dipendenze con: pip install openpyxl reportlab")

In [ ]:
# Percorsi
BASE_DIR = Path(os.getcwd())
EXCEL_FILE = 'Pianificazione_Corsi_2026.xlsx'
PDF_DIR = 'stampe_pdf'

# Crea directory PDF se non esiste
Path(PDF_DIR).mkdir(exist_ok=True)

# Configurazione formatori e aule
FORMATORI = ['CL', 'MC', 'LD', 'EP', 'IP', 'FB', 'GZ', 'DC']
FORMATORI_TEST = ['URS', 'NIC', 'MIT', 'MON', 'WER']
AULE = ['103', '103a', '108', '110', 'UFF']
ATTIVITA_ESTERNE = ['RIUNIONE', 'FORMAZIONE', 'CONSULENZA', 'AUDIT', 'ALTRO']

# Aule con attività compatibili
AULE_ATTIVITA = {
    '103': ['AULA', 'DIGI', 'CV', 'CS', 'RA', 'TT', 'TI'],
    '103a': ['AULA', 'DIGI', 'CV', 'CS', 'RA', 'TT', 'TI'],
    '108': ['AULA', 'CV', 'CS', 'RA'],
    '110': ['AULA', 'CV', 'CS', 'RA'],
    'UFF': ['UFF', 'COL', 'C']
}

# Festività 2026
FESTIVITA = [
    datetime(2026, 1, 1), datetime(2026, 1, 6), datetime(2026, 4, 6),
    datetime(2026, 4, 25), datetime(2026, 5, 1), datetime(2026, 6, 2),
    datetime(2026, 8, 15), datetime(2026, 11, 1), datetime(2026, 12, 8),
    datetime(2026, 12, 25), datetime(2026, 12, 26),
    # Ferie aziendali
    datetime(2026, 8, 10), datetime(2026, 8, 11), datetime(2026, 8, 12),
    datetime(2026, 8, 13), datetime(2026, 8, 14),
]

# Colori per Excel
COLOR_HEADER = 'D9E1F2'
COLOR_MORNING = 'E2EFDA'
COLOR_AFTERNOON = 'FFF2CC'

print(f"📁 Directory lavoro: {BASE_DIR}")
print(f"📄 File Excel: {EXCEL_FILE}")
print(f"📂 Directory PDF: {PDF_DIR}")

In [ ]:
def apri_file(filepath):
    """Apre un file con l'applicazione predefinita del sistema"""
    try:
        if platform.system() == 'Windows':
            os.startfile(filepath)
        elif platform.system() == 'Darwin':  # macOS
            subprocess.run(['open', filepath])
        else:  # Linux
            subprocess.run(['xdg-open', filepath])
        print(f"✅ File aperto: {filepath}")
    except Exception as e:
        print(f"❌ Errore: {e}")

def apri_cartella(dirpath):
    """Apre una cartella nel file manager"""
    try:
        if platform.system() == 'Windows':
            os.startfile(dirpath)
        elif platform.system() == 'Darwin':
            subprocess.run(['open', dirpath])
        else:
            subprocess.run(['xdg-open', dirpath])
        print(f"✅ Cartella aperta: {dirpath}")
    except Exception as e:
        print(f"❌ Errore: {e}")

def lista_pdf():
    """Elenca tutti i PDF generati"""
    pdf_path = Path(PDF_DIR)
    if not pdf_path.exists():
        print("📂 La cartella PDF non esiste ancora.")
        return []
    
    pdfs = list(pdf_path.glob('*.pdf'))
    if pdfs:
        print(f"\\n📄 PDF disponibili ({len(pdfs)}):") 
        for pdf in sorted(pdfs):
            size = pdf.stat().st_size
            size_str = f"{size // 1024} KB" if size < 1024 * 1024 else f"{size // (1024 * 1024)} MB"
            print(f"   - {pdf.name} ({size_str})")
    else:
        print("📂 Nessun PDF trovato.")
    return pdfs

print("✅ Funzioni utility caricate!")

In [ ]:
# Aggiungi la directory corrente al path di Python
import sys
if str(BASE_DIR) not in sys.path:
    sys.path.insert(0, str(BASE_DIR))

# Prova a importare i moduli del progetto
try:
    from crea_pianificazione_smart import crea_pianificazione_excel
    from genera_stampe_pdf import (
        carica_dati_excel,
        genera_report_aule,
        genera_report_formatori,
        genera_report_corsi,
        genera_report_settimanale
    )
    from genera_stampe_pdf_filtrati import (
        genera_report_aule_settimane,
        genera_report_formatore_periodo,
        genera_report_corso_specifico
    )
    print("✅ Tutti i moduli del progetto importati con successo!")
    MODULI_DISPONIBILI = True
except ImportError as e:
    print(f"⚠️  Alcuni moduli non sono disponibili: {e}")
    print("   Il notebook funzionerà con funzionalità limitate.")
    MODULI_DISPONIBILI = False

In [ ]:
if Path(EXCEL_FILE).exists():
    apri_file(EXCEL_FILE)
else:
    print(f"❌ File non trovato: {EXCEL_FILE}")

In [ ]:
if not MODULI_DISPONIBILI:
    print("❌ Moduli non disponibili.")
elif not Path(EXCEL_FILE).exists():
    print(f"❌ File non trovato: {EXCEL_FILE}")
    print("   Crea prima il file Excel eseguendo la sezione 5.")
else:
    print("\\n" + "="*70)
    print("🚀 GENERAZIONE COMPLETA PDF")
    print("="*70 + "\\n")
    
    dati = carica_dati_excel(EXCEL_FILE)
    
    if not dati:
        print("⚠️  Nessun dato trovato. Compila il file Excel prima di generare i PDF.")
    else:
        # 1. Prenotazione Aule
        print("📄 1/4 - Prenotazione Aule...")
        genera_report_aule(dati, PDF_DIR)
        print("✅ Completato!\\n")
        
        # 2. Programma Formatori
        print("📄 2/4 - Programma Formatori...")
        genera_report_formatori(dati, PDF_DIR)
        print("✅ Completato!\\n")
        
        # 3. Programma Corsi
        print("📄 3/4 - Programma Corsi...")
        genera_report_corsi(dati, PDF_DIR)
        print("✅ Completato!\\n")
        
        # 4. Report Settimanali
        print("📄 4/4 - Report Settimanali...")
        genera_report_settimanale(dati, PDF_DIR)
        print("✅ Completato!\\n")
        
        print("="*70)
        print("🎉 GENERAZIONE COMPLETATA!")
        print("="*70 + "\\n")
        
        lista_pdf()
        
        print("\\n🔍 Apertura cartella PDF...")
        apri_cartella(PDF_DIR)

---

## 📚 Note e Istruzioni

### Come usare questo notebook:

1. **Prerequisiti**: Assicurati di avere installato:
   - `pip install openpyxl reportlab`

2. **Creazione Excel**: Esegui la sezione 5 per creare il file `Pianificazione_Corsi_2026.xlsx`

3. **Compila dati**: Apri il file Excel e inserisci i dati di pianificazione

4. **Genera PDF**: Esegui la sezione 6 per generare tutti i PDF

5. **Gestione file**: Usa la sezione 7 per visualizzare ed aprire i file generati

### Struttura file:

```
Directory di lavoro/
├── Pianificazione_Corsi_2026.xlsx    (file di pianificazione)
├── stampe_pdf/                        (cartella PDF generati)
│   ├── Prenotazione_Aule_2026.pdf
│   ├── Programma_Formatore_*.pdf
│   ├── Programma_Corso_*.pdf
│   └── Piano_Settimanale_*.pdf
├── crea_pianificazione_smart.py       (modulo creazione Excel)
├── genera_stampe_pdf.py               (modulo generazione PDF)
├── genera_stampe_pdf_filtrati.py      (modulo PDF filtrati)
└── pianificazione_completa.ipynb      (questo notebook)
```

### ✅ Vantaggi di questo notebook:

- **Autonomo**: Funziona senza server web
- **Portabile**: Può essere eseguito su qualsiasi sistema con Python e Jupyter
- **Completo**: Replica tutte le funzionalità dell'app web
- **Flessibile**: Puoi modificare le celle per personalizzare il comportamento

---

**🎯 Il notebook è pronto all'uso!**

In [ ]:
apri_cartella(PDF_DIR)

### Apri cartella PDF

In [ ]:
lista_pdf()

---

## 7. 📂 GESTIONE FILE

### Elenca PDF generati

---

## 6. 📄 GENERAZIONE PDF

### 🚀 Workflow Completo - Genera tutti i PDF

Esegui questa cella per generare tutti i PDF in una volta:

### 🔍 Apri il file Excel

In [ ]:
if not MODULI_DISPONIBILI:
    print("❌ Moduli non disponibili. Impossibile creare il file Excel.")
else:
    if Path(EXCEL_FILE).exists():
        risposta = input(f"⚠️  Il file {EXCEL_FILE} esiste già. Sovrascrivere? (s/n): ")
        if risposta.lower() != 's':
            print("❌ Operazione annullata.")
        else:
            print("\\n🔄 Creazione file Excel in corso...")
            crea_pianificazione_excel()
            print(f"\\n✅ File Excel creato: {EXCEL_FILE}")
    else:
        print("\\n🔄 Creazione file Excel in corso...")
        crea_pianificazione_excel()
        print(f"\\n✅ File Excel creato: {EXCEL_FILE}")

---

## 5. 📊 CREAZIONE FILE EXCEL

Crea il file Excel di pianificazione con tutti i giorni lavorativi del 2026:

## 4. Import Moduli del Progetto

Importiamo i moduli del progetto. Se non sono disponibili, il notebook può comunque funzionare con le funzioni integrate:

## 3. Funzioni Utility

Funzioni per aprire file e cartelle:

## 2. Configurazione e Costanti

## 1. Import delle Librerie

Importiamo tutte le librerie necessarie (assumiamo che siano già installate):

## 1. Installazione Dipendenze

Esegui questa cella per installare tutti i pacchetti necessari:

In [1]:
# Installa le dipendenze necessarie
import sys
!{sys.executable} -m pip install openpyxl reportlab -q

You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


## 2. Import delle Librerie

In [ ]:
import os
import subprocess
import platform
from datetime import datetime, timedelta
from pathlib import Path

# Import dei moduli del progetto
from crea_pianificazione_smart import crea_pianificazione_excel
from genera_stampe_pdf import (
    carica_dati_excel,
    genera_report_aule,
    genera_report_formatori,
    genera_report_corsi,
    genera_report_settimanale
)
from genera_stampe_pdf_filtrati import (
    genera_report_aule_settimane,
    genera_report_formatore_periodo,
    genera_report_corso_specifico
)

print("✅ Librerie importate con successo!")

## 3. Configurazione Percorsi

In [ ]:
# Determina la directory di lavoro
BASE_DIR = Path(os.getcwd())
EXCEL_FILE = BASE_DIR / 'Pianificazione_Corsi_2026.xlsx'
PDF_DIR = BASE_DIR / 'stampe_pdf'

# Crea la directory per i PDF se non esiste
PDF_DIR.mkdir(exist_ok=True)

print(f"📁 Directory di lavoro: {BASE_DIR}")
print(f"📄 File Excel: {EXCEL_FILE}")
print(f"📂 Directory PDF: {PDF_DIR}")

## 4. Funzioni Utility

In [ ]:
def apri_file(filepath):
    """Apre un file con l'applicazione predefinita del sistema"""
    try:
        if platform.system() == 'Windows':
            os.startfile(filepath)
        elif platform.system() == 'Darwin':  # macOS
            subprocess.run(['open', filepath])
        else:  # Linux
            subprocess.run(['xdg-open', filepath])
        print(f"✅ File aperto: {filepath}")
    except Exception as e:
        print(f"❌ Errore nell'apertura del file: {e}")

def apri_cartella(dirpath):
    """Apre una cartella nel file manager del sistema"""
    try:
        if platform.system() == 'Windows':
            os.startfile(dirpath)
        elif platform.system() == 'Darwin':  # macOS
            subprocess.run(['open', dirpath])
        else:  # Linux
            subprocess.run(['xdg-open', dirpath])
        print(f"✅ Cartella aperta: {dirpath}")
    except Exception as e:
        print(f"❌ Errore nell'apertura della cartella: {e}")

def lista_pdf():
    """Elenca tutti i PDF generati"""
    if not PDF_DIR.exists():
        print("📂 La cartella PDF non esiste ancora.")
        return []
    
    pdfs = list(PDF_DIR.glob('*.pdf'))
    if pdfs:
        print(f"\n📄 PDF disponibili ({len(pdfs)}):")
        for pdf in sorted(pdfs):
            size = pdf.stat().st_size
            size_str = f"{size // 1024} KB" if size < 1024 * 1024 else f"{size // (1024 * 1024)} MB"
            print(f"   - {pdf.name} ({size_str})")
    else:
        print("📂 Nessun PDF trovato.")
    return pdfs

print("✅ Funzioni utility caricate!")

---
## 5. 📊 CREAZIONE FILE EXCEL

Crea il file Excel di pianificazione con:
- Foglio 2026 con tutti i giorni lavorativi
- Foglio Assumptions con configurazioni
- Data validation per formatori, aule, attività
- Conditional formatting per duplicati

In [ ]:
# Verifica se il file esiste già
if EXCEL_FILE.exists():
    risposta = input(f"⚠️  Il file {EXCEL_FILE.name} esiste già. Sovrascrivere? (s/n): ")
    if risposta.lower() != 's':
        print("❌ Operazione annullata.")
    else:
        print("\n🔄 Creazione file Excel in corso...")
        crea_pianificazione_excel()
        print(f"\n✅ File Excel creato con successo: {EXCEL_FILE}")
else:
    print("\n🔄 Creazione file Excel in corso...")
    crea_pianificazione_excel()
    print(f"\n✅ File Excel creato con successo: {EXCEL_FILE}")

### 🔍 Apri il file Excel per compilarlo

In [ ]:
if EXCEL_FILE.exists():
    apri_file(str(EXCEL_FILE))
else:
    print(f"❌ File non trovato: {EXCEL_FILE}")

---
## 6. 📄 GENERAZIONE PDF

### 6.1 Prenotazione Aule (Completo)

In [ ]:
# Genera PDF prenotazione aule per tutto l'anno
if not EXCEL_FILE.exists():
    print(f"❌ File non trovato: {EXCEL_FILE}")
else:
    print("\n📄 Generazione PDF Prenotazione Aule...\n")
    dati = carica_dati_excel(str(EXCEL_FILE))
    
    if dati:
        genera_report_aule(dati, str(PDF_DIR))
        print("\n✅ PDF Aule generato con successo!")
    else:
        print("⚠️  Nessun dato trovato nel file Excel.")

### 6.2 Prenotazione Aule (Filtrato per settimane)

In [ ]:
# Genera PDF prenotazione aule per settimane specifiche
# Esempio: settimane 1, 2, 3
settimane = [1, 2, 3]  # Modifica questa lista per selezionare le settimane desiderate

if not EXCEL_FILE.exists():
    print(f"❌ File non trovato: {EXCEL_FILE}")
else:
    print(f"\n📄 Generazione PDF Aule per settimane {settimane}...\n")
    dati = carica_dati_excel(str(EXCEL_FILE))
    
    if dati:
        genera_report_aule_settimane(dati, str(PDF_DIR), settimane)
        print("\n✅ PDF Aule filtrato generato con successo!")
    else:
        print("⚠️  Nessun dato trovato nel file Excel.")

### 6.3 Programma Formatori (Tutti)

In [ ]:
# Genera PDF per tutti i formatori
if not EXCEL_FILE.exists():
    print(f"❌ File non trovato: {EXCEL_FILE}")
else:
    print("\n📄 Generazione PDF Programma Formatori...\n")
    dati = carica_dati_excel(str(EXCEL_FILE))
    
    if dati:
        genera_report_formatori(dati, str(PDF_DIR))
        print("\n✅ PDF Formatori generati con successo!")
    else:
        print("⚠️  Nessun dato trovato nel file Excel.")

### 6.4 Programma Formatore (Specifico con periodo)

In [ ]:
# Genera PDF per un formatore specifico in un periodo
formatore = 'CL'  # Cambia con il nome del formatore desiderato
data_inizio = datetime(2026, 1, 1)  # Data di inizio
data_fine = datetime(2026, 12, 31)  # Data di fine

if not EXCEL_FILE.exists():
    print(f"❌ File non trovato: {EXCEL_FILE}")
else:
    print(f"\n📄 Generazione PDF per {formatore} ({data_inizio.strftime('%d/%m/%Y')} - {data_fine.strftime('%d/%m/%Y')})...\n")
    dati = carica_dati_excel(str(EXCEL_FILE))
    
    if dati:
        genera_report_formatore_periodo(formatore, data_inizio, data_fine, str(EXCEL_FILE), str(PDF_DIR))
        print(f"\n✅ PDF per {formatore} generato con successo!")
    else:
        print("⚠️  Nessun dato trovato nel file Excel.")

### 6.5 Programma Corsi (Tutti)

In [ ]:
# Genera PDF per tutti i corsi
if not EXCEL_FILE.exists():
    print(f"❌ File non trovato: {EXCEL_FILE}")
else:
    print("\n📄 Generazione PDF Programma Corsi...\n")
    dati = carica_dati_excel(str(EXCEL_FILE))
    
    if dati:
        genera_report_corsi(dati, str(PDF_DIR))
        print("\n✅ PDF Corsi generati con successo!")
    else:
        print("⚠️  Nessun dato trovato nel file Excel.")

### 6.6 Programma Corso (Specifico)

In [ ]:
# Genera PDF per un corso specifico
corso = '1a'  # Cambia con il nome del corso desiderato

if not EXCEL_FILE.exists():
    print(f"❌ File non trovato: {EXCEL_FILE}")
else:
    print(f"\n📄 Generazione PDF per corso {corso}...\n")
    dati = carica_dati_excel(str(EXCEL_FILE))
    
    if dati:
        genera_report_corso_specifico(corso, str(EXCEL_FILE), str(PDF_DIR))
        print(f"\n✅ PDF per corso {corso} generato con successo!")
    else:
        print("⚠️  Nessun dato trovato nel file Excel.")

### 6.7 Report Settimanali

In [ ]:
# Genera PDF report settimanali
if not EXCEL_FILE.exists():
    print(f"❌ File non trovato: {EXCEL_FILE}")
else:
    print("\n📄 Generazione Report Settimanali...\n")
    dati = carica_dati_excel(str(EXCEL_FILE))
    
    if dati:
        genera_report_settimanale(dati, str(PDF_DIR))
        print("\n✅ Report settimanali generati con successo!")
    else:
        print("⚠️  Nessun dato trovato nel file Excel.")

---
## 7. 📂 GESTIONE FILE

### 7.1 Elenca tutti i PDF generati

In [ ]:
pdfs = lista_pdf()

### 7.2 Apri la cartella PDF

In [ ]:
apri_cartella(str(PDF_DIR))

### 7.3 Apri un PDF specifico

In [ ]:
# Apri un PDF specifico (cambia il nome del file)
pdf_file = PDF_DIR / 'Prenotazione_Aule_2026.pdf'

if pdf_file.exists():
    apri_file(str(pdf_file))
else:
    print(f"❌ File non trovato: {pdf_file}")
    print("\n📄 PDF disponibili:")
    lista_pdf()

---
## 8. 🎯 WORKFLOW COMPLETO

Esegui questa cella per generare tutti i PDF in una volta sola:

In [ ]:
def workflow_completo():
    """Genera tutti i PDF in una volta sola"""
    if not EXCEL_FILE.exists():
        print(f"❌ File non trovato: {EXCEL_FILE}")
        print("   Crea prima il file Excel eseguendo la cella nella sezione 5.")
        return
    
    print("\n" + "="*70)
    print("🚀 INIZIO GENERAZIONE COMPLETA PDF")
    print("="*70 + "\n")
    
    dati = carica_dati_excel(str(EXCEL_FILE))
    
    if not dati:
        print("⚠️  Nessun dato trovato nel file Excel.")
        print("   Compila il file Excel prima di generare i report.")
        return
    
    # 1. Prenotazione Aule
    print("\n📄 1/4 - Generazione PDF Prenotazione Aule...")
    genera_report_aule(dati, str(PDF_DIR))
    print("✅ Completato!")
    
    # 2. Programma Formatori
    print("\n📄 2/4 - Generazione PDF Programma Formatori...")
    genera_report_formatori(dati, str(PDF_DIR))
    print("✅ Completato!")
    
    # 3. Programma Corsi
    print("\n📄 3/4 - Generazione PDF Programma Corsi...")
    genera_report_corsi(dati, str(PDF_DIR))
    print("✅ Completato!")
    
    # 4. Report Settimanali
    print("\n📄 4/4 - Generazione Report Settimanali...")
    genera_report_settimanale(dati, str(PDF_DIR))
    print("✅ Completato!")
    
    print("\n" + "="*70)
    print("🎉 GENERAZIONE COMPLETA TERMINATA CON SUCCESSO!")
    print("="*70 + "\n")
    
    # Elenca i PDF generati
    lista_pdf()
    
    # Apri la cartella PDF
    print("\n🔍 Apertura cartella PDF...")
    apri_cartella(str(PDF_DIR))

# Esegui il workflow completo
workflow_completo()

---
## 9. 📊 STATISTICHE E INFORMAZIONI

Visualizza informazioni sul file Excel e sui PDF generati:

In [ ]:
def mostra_statistiche():
    """Mostra statistiche sul progetto"""
    print("\n" + "="*70)
    print("📊 STATISTICHE PROGETTO")
    print("="*70 + "\n")
    
    # File Excel
    if EXCEL_FILE.exists():
        size = EXCEL_FILE.stat().st_size
        size_str = f"{size // 1024} KB"
        mod_time = datetime.fromtimestamp(EXCEL_FILE.stat().st_mtime)
        print(f"📄 File Excel: {EXCEL_FILE.name}")
        print(f"   Dimensione: {size_str}")
        print(f"   Ultima modifica: {mod_time.strftime('%d/%m/%Y %H:%M:%S')}")
        
        # Carica dati per statistiche
        try:
            dati = carica_dati_excel(str(EXCEL_FILE))
            if dati:
                print(f"   Righe caricate: {len(dati)}")
                
                # Conta formatori unici
                formatori = set()
                for riga in dati:
                    for perc_label, perc_dati in riga['percorsi']:
                        f1 = perc_dati.get('formatore1')
                        f2 = perc_dati.get('formatore2')
                        if f1:
                            formatori.add(f1)
                        if f2:
                            formatori.add(f2)
                print(f"   Formatori unici: {len(formatori)}")
        except Exception as e:
            print(f"   ⚠️  Errore nel caricamento dati: {e}")
    else:
        print(f"❌ File Excel non trovato: {EXCEL_FILE}")
    
    print()
    
    # PDF generati
    if PDF_DIR.exists():
        pdfs = list(PDF_DIR.glob('*.pdf'))
        if pdfs:
            total_size = sum(pdf.stat().st_size for pdf in pdfs)
            total_size_str = f"{total_size // 1024} KB" if total_size < 1024 * 1024 else f"{total_size // (1024 * 1024)} MB"
            print(f"📂 Cartella PDF: {PDF_DIR}")
            print(f"   PDF generati: {len(pdfs)}")
            print(f"   Dimensione totale: {total_size_str}")
        else:
            print(f"📂 Cartella PDF: {PDF_DIR}")
            print("   Nessun PDF generato ancora.")
    else:
        print("📂 Cartella PDF non esistente.")
    
    print("\n" + "="*70 + "\n")

mostra_statistiche()

---
## 📚 Note e Istruzioni

### Come usare questo notebook:

1. **Installazione**: Esegui la cella 1 per installare le dipendenze
2. **Import**: Esegui le celle 2-4 per caricare le librerie e configurare i percorsi
3. **Creazione Excel**: Esegui la sezione 5 per creare il file Excel
4. **Compila Excel**: Apri il file Excel e inserisci i dati di pianificazione
5. **Genera PDF**: Esegui le celle nella sezione 6 per generare i PDF desiderati
6. **Workflow completo**: Usa la cella nella sezione 8 per generare tutti i PDF in una volta

### Personalizzazione:

- Per generare PDF per formatori/corsi specifici, modifica le variabili nelle celle corrispondenti
- Per filtrare per periodo, modifica `data_inizio` e `data_fine`
- Per filtrare per settimane, modifica la lista `settimane`

### Struttura dei file:

```
Pianificazione_Windows/
├── Pianificazione_Corsi_2026.xlsx    (file di pianificazione)
├── stampe_pdf/                        (cartella con i PDF generati)
│   ├── Prenotazione_Aule_2026.pdf
│   ├── Programma_Formatore_*.pdf
│   ├── Programma_Corso_*.pdf
│   └── Piano_Settimanale_*.pdf
└── pianificazione_completa.ipynb      (questo notebook)
```

---

**🎯 Questo notebook replica completamente tutte le funzionalità dell'applicazione web!**